In [3]:
import torchvision

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder ,DatasetFolder,ImageFolder
from torch.utils.data import ConcatDataset, DataLoader, Subset,Dataset
import torch
from tqdm.auto import tqdm
from PIL import Image
import torchvision





trainTfm =transforms.Compose([transforms.Resize((128,128)),
  transforms.RandomRotation(45),
  transforms.RandomHorizontalFlip(0.5),
  transforms.RandomVerticalFlip(0.5),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
testTfm =transforms.Compose([transforms.Resize((128,128)), 
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])




ModuleNotFoundError: No module named 'wgModular'

In [2]:
batch_size = 128
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trainSet = ImageFolder("./data/food-11/training/labeled/",transform=trainTfm)
unlabelSet = ImageFolder("./data/food-11/training/unlabeled/",transform=trainTfm)
validSet = ImageFolder("./data/food-11/validation/",transform=testTfm)
testSet = ImageFolder("./data/food-11/testing/",transform=testTfm)

NameError: name 'trainTfm' is not defined

In [ ]:
aa = torch.Tensor([1,2,3,4,5,6,7,8,9,10,11,12]).reshape(1,2,2,3)

In [ ]:
vgg = torchvision.models.vgg11_bn()
layers =vgg.features
fc = vgg.classifier

In [ ]:
fc